### Topic - Retrieval
##### Using tools to get data from the context

In [1]:
import json
import os

from openai import OpenAI
from pydantic import BaseModel, Field

In [2]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## 🛠️ Function Calling

**Function Calling**: Lets the AI call your backend functions with structured arguments based on the user's request.

- Allows dynamic interaction between AI and external tools or APIs.
- Useful for tasks like retrieving data, triggering actions, or generating responses based on real-time inputs.

### 🔧 Example: Knowledge Base Search

This example demonstrates how an LLM can answer questions by calling a custom `search_kb` function that reads from a local JSON knowledge base.

#### 🔹 Steps Overview

1. **Define a function tool**  
   A function `search_kb(question)` is exposed to the model with a clear description and parameter schema.

2. **Model decides to use the tool**  
   The model evaluates the prompt and determines whether to call the function (e.g., for "What is the return policy?").

3. **Tool execution**  
   If the tool is selected, the backend extracts the tool name and arguments, runs the function, and returns the result.

4. **Return final answer**  
   The result from the tool is appended to the messages, and the model is called again to generate the final, user-facing response.

5. **Fallback case**  
   If the user asks something irrelevant (e.g., "What's the weather in Tokyo?"), the model may choose not to call the function.


---
##### Define the knowledge base retrieval tool


In [3]:
def search_kb(question: str):
    """
    Load the whole knowledge base from the JSON file.
    (This is a mock function for demonstration purposes, we don't search)
    """
    with open("kb.json", "r") as f:
        return json.load(f)


#### Step 1: Call model with search_kb tool defined


In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_kb",
            "description": "Get the answer to the user's question from the knowledge base.",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {"type": "string"},
                },
                "required": ["question"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
]

In [12]:
system_prompt = "You are a helpful assistant that answers questions from the knowledge base about our e-commerce store."

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "policy to reurn?"},
]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)



#### Step 2: Model decides to call function(s)


In [13]:

completion.model_dump()

{'id': 'chatcmpl-BU5bF7R6zx0SWG1xJvJ6omLWk9Ing',
 'choices': [{'finish_reason': 'tool_calls',
   'index': 0,
   'logprobs': None,
   'message': {'content': None,
    'refusal': None,
    'role': 'assistant',
    'annotations': [],
    'audio': None,
    'function_call': None,
    'tool_calls': [{'id': 'call_yKE7XdoiAhG6OQ9MvTOQmimL',
      'function': {'arguments': '{"question":"return policy"}',
       'name': 'search_kb'},
      'type': 'function'}]}}],
 'created': 1746511065,
 'model': 'gpt-4o-2024-08-06',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_f5bdcc3276',
 'usage': {'completion_tokens': 17,
  'prompt_tokens': 73,
  'total_tokens': 90,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}}

#### Step 3: Execute search_kb function

In [14]:
def call_function(name, args):
    if name == "search_kb":
        return search_kb(**args)


In [15]:
for tool_call in completion.choices[0].message.tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    messages.append(completion.choices[0].message)

    result = call_function(name, args)
    messages.append(
        {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
    )


In [16]:
name

'search_kb'

In [17]:
args

{'question': 'return policy'}

In [11]:
messages

[{'role': 'system',
  'content': 'You are a helpful assistant that answers questions from the knowledge base about our e-commerce store.'},
 {'role': 'user', 'content': 'What is the return policy?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ZVBZhUzVejz887x4EbkV9Pfs', function=Function(arguments='{"question":"What is the return policy?"}', name='search_kb'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_ZVBZhUzVejz887x4EbkV9Pfs',
  'content': '{"records": [{"id": 1, "question": "What is the return policy?", "answer": "Items can be returned within 30 days of purchase with original receipt. Refunds will be processed to the original payment method within 5-7 business days."}, {"id": 2, "question": "Do you ship internationally?", "answer": "Yes, we ship to over 50 countries worldwide. International shipping typically takes 7-14 business days and cost

In [18]:
result

{'records': [{'id': 1,
   'question': 'What is the return policy?',
   'answer': 'Items can be returned within 30 days of purchase with original receipt. Refunds will be processed to the original payment method within 5-7 business days.'},
  {'id': 2,
   'question': 'Do you ship internationally?',
   'answer': 'Yes, we ship to over 50 countries worldwide. International shipping typically takes 7-14 business days and costs vary by destination. Please note that customs fees may apply.'},
  {'id': 3,
   'question': 'What payment methods do you accept?',
   'answer': 'We accept Visa, Mastercard, American Express, PayPal, and Apple Pay. All payments are processed securely through our encrypted payment system.'}]}

#### Step 4: Supply result and call model again

In [19]:
class KBResponse(BaseModel):
    answer: str = Field(description="The answer to the user's question.")
    source: int = Field(description="The record id of the answer.")

In [20]:
completion_2 = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    response_format=KBResponse,
)


#### Step 5: Check model response

In [21]:
final_response = completion_2.choices[0].message.parsed

In [22]:
final_response

KBResponse(answer='Items can be returned within 30 days of purchase with original receipt. Refunds will be processed to the original payment method within 5-7 business days.', source=1)

In [23]:
final_response.answer

'Items can be returned within 30 days of purchase with original receipt. Refunds will be processed to the original payment method within 5-7 business days.'

In [24]:

final_response.source


1

##### Question that doesn't trigger the tool

In [25]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "What is the weather in Tokyo?"},
]

completion_3 = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

In [26]:
completion_3.model_dump()

{'id': 'chatcmpl-BU5dv5KsZZoD8JY8kaob8Z3ZHLlDn',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': 'I am unable to access real-time weather information. You may want to check a dedicated weather service or app for the current weather in Tokyo.',
    'refusal': None,
    'role': 'assistant',
    'annotations': [],
    'audio': None,
    'function_call': None,
    'tool_calls': None,
    'parsed': None}}],
 'created': 1746511231,
 'model': 'gpt-4o-2024-08-06',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_f5bdcc3276',
 'usage': {'completion_tokens': 30,
  'prompt_tokens': 75,
  'total_tokens': 105,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}}

In [27]:
completion_3.choices[0].message.content

'I am unable to access real-time weather information. You may want to check a dedicated weather service or app for the current weather in Tokyo.'

> 📖 More details of Function Calling on the [OpenAI Docs](https://platform.openai.com/docs/guides/function-calling)

